In [2]:
import pandas as pd
import plotly.express as px
import os
import sys
import statsmodels.api as sm
import numpy as np
import glob

In [25]:
dfs = []
for file in glob.glob("../testing/0_1GB/*.counted"):
    contig = file.rsplit("/",1)[1].split(".counted")[0]

    df = pd.read_csv(file, sep=" ", names=["mer","mer_count"])
    df_res = df.describe().transpose().reset_index(drop=True)
    df_res["Contig"] = contig
    df_res["count_nonzero"] = len(df.query("mer_count > 0"))
    df_res["count_unique"] = len(df.mer.drop_duplicates())
    df_res["count_unique_nonzero"] = len(df.query("mer_count > 0").mer.drop_duplicates())
    df_res.rename(columns={"50%":"Depth median", "mean":"Depth avg"}, inplace=True)
    df_out = df_res[["Contig"]+[x for x in df_res.columns if x != "Contig"]]
    dfs.append(df_out)
df_summation = pd.concat(dfs)
df_summation.to_csv("../testing/0_1GB/summation.tsv", sep="\t", index=False)

In [16]:
df

,mer,mer_count
0,TTAATCGTCTTTATCAATTAA,3
1,ATTAATTGATAAAGACGATTA,3
2,AATCGTCTTTATCAATTAATA,3
3,ATCGTCTTTATCAATTAATAC,3
4,AGTATTAATTGATAAAGACGA,3
...,...,...
20727,ATGGACGTCAGTCCTCAGAAA,4
20728,GTTTCTGAGGACTGACGTCCA,4
20729,AGTTTCTGAGGACTGACGTCC,4
20730,GACGTCAGTCCTCAGAAACTA,4


In [22]:
file = "../testing/0_1GB/NC_014328.1.region003.counted"


,Contig,count,mean,std,min,25%,50%,75%,max,count_nonzero,count_unique,count_unique_nonzero
0,NC_014328.1.region003,20732.0,3.448871,2.133014,0.0,2.0,3.0,5.0,15.0,20017,20725,20010


In [27]:
os.getcwd()

OSError: [Errno 107] Transport endpoint is not connected

In [2]:
# import count file:
df = pd.read_csv("../testing/bgc_counts/NC_014328.1.region001.csv")

In [6]:
df_res = df.describe().transpose().reset_index(drop=True)
df_res

,count,mean,std,min,25%,50%,75%,max
0,18132.0,2.881811,2.399439,0.0,2.0,3.0,4.0,111.0


In [13]:
df_res["Contig"] = "NC_014328.1.region001"
df_res["count_nonzero"] = len(df.query("count > 0"))
df_res.rename(columns={"50%":"Depth median", "mean":"Depth avg"})

,count,Depth avg,std,min,25%,Depth median,75%,max,Contig,count_zero,count_nonzero
0,18132.0,2.881811,2.399439,0.0,2.0,3.0,4.0,111.0,NC_014328.1.region001,16997,16997


In [33]:
# Analyse counts:
fig = px.histogram(df, x="count", histnorm='probability density')
fig

In [22]:
s = df["count"].values
X = np.ones_like(s)

model = sm.NegativeBinomial(s,X).fit(start_params=[np.median(s),1])
model.summary()

Optimization terminated successfully.
         Current function value: 1.996314
         Iterations: 12
         Function evaluations: 14
         Gradient evaluations: 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                     NegativeBinomial Regression Results                      
==============================================================================
Dep. Variable:                      y   No. Observations:                18132
Model:               NegativeBinomial   Df Residuals:                    18131
Method:                           MLE   Df Model:                            0
Date:                Wed, 21 Sep 2022   Pseudo R-squ.:               8.388e-12
Time:                        11:38:39   Log-Likelihood:                -36197.
converged:                       True   LL-Null:                       -36197.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0584      0.005    206.836      0.000       1.048       1.068
alpha          0.1278      0.004     28.679      0.000       0.119       0.137
==============================================================================
"""

In [23]:
import seaborn as sns
from scipy.stats import nbinom

In [25]:
mu = np.exp(model.params[0])
p = 1/(1+np.exp(model.params[0])*model.params[1])
n = np.exp(model.params[0])*p/(1-p)
mu, p, n

(2.8818114333865745, 0.7308480858331274, 7.825195582732467)

In [27]:
x_plot = np.linspace(0, 20, 20)
y = nbinom.pmf(x_plot, n, p)

In [34]:
import plotly.graph_objects as go
fig.add_trace(
    go.Scatter(x=x_plot, y=y)
)